In [61]:
# import những thư viện cần thiết
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
# Tiền xử lý
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Xây dựng mô hình
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [62]:
# Đọc dữ liệu từ file csv
df = pd.read_csv('../data/cleaned_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5309 entries, 0 to 5308
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Khu vực tuyển         5309 non-null   object 
 1   Thời gian thử việc    5309 non-null   float64
 2   Cấp bậc               5309 non-null   object 
 3   Yêu cầu giới tính     5309 non-null   object 
 4   Số lượng tuyển        5309 non-null   int64  
 5   Hình thức làm việc    5309 non-null   object 
 6   Yêu cầu bằng cấp      5309 non-null   object 
 7   Yêu cầu kinh nghiệm   5309 non-null   object 
 8   Ngành nghề            5309 non-null   object 
 9   Quy mô công ty        5309 non-null   object 
 10  Loại công ty          5309 non-null   object 
 11  Mức lương thấp nhất   5309 non-null   float64
 12  Mức lương cao nhất    5309 non-null   float64
 13  Mức lương trung bình  5309 non-null   float64
 14  Tuổi thấp nhất        5309 non-null   float64
 15  Tuổi cao nhất        

In [63]:
df[25:30]

,Khu vực tuyển,Thời gian thử việc,Cấp bậc,Yêu cầu giới tính,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Quy mô công ty,Loại công ty,Mức lương thấp nhất,Mức lương cao nhất,Mức lương trung bình,Tuổi thấp nhất,Tuổi cao nhất,Tuổi trung bình
25,Bình Dương,1.0,Chuyên viên- nhân viên,Không yêu cầu,2,Toàn thời gian cố định,Cao đẳng,Dưới 1 năm,Sản xuất - Lắp ráp - Chế biến/Hành chính - Thư...,150 - 300 người,Công ty trách nhiệm hữu hạn,10.0,15.0,12.5,23.0,41.0,32.0
26,TP.HCM,2.8,Chuyên viên- nhân viên,Nữ,2,Toàn thời gian cố định,Đại học,Chưa có kinh nghiệm,Hành chính - Thư ký/Tài chính - Đầu tư - Chứng...,Dưới 10 người,Công ty trách nhiệm hữu hạn,8.0,12.0,10.0,23.0,37.0,30.0
27,"Khánh Hòa, Đắk Nông, Kon Tum, Gia Lai, Quảng N...",1.8,Chuyên viên- nhân viên,Không yêu cầu,10,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,Vận Tải - Lái xe - Giao nhận/Lao động phổ thôn...,150 - 300 người,Công ty trách nhiệm hữu hạn,8.0,15.0,11.5,22.0,37.0,29.5
28,Đồng Nai,2.0,Chuyên viên- nhân viên,Không yêu cầu,1,Toàn thời gian cố định,Cao đẳng,2 năm,Hành chính - Thư ký/Nhân sự,150 - 300 người,Công ty trách nhiệm hữu hạn,13.0,16.0,14.5,22.0,40.0,31.0
29,TP.HCM,2.2,Chuyên viên- nhân viên,Không yêu cầu,2,Toàn thời gian cố định,Trung cấp,1 năm,Biên phiên dịch/Hành chính - Thư ký/Thông tin ...,10 - 150 người,Công ty trách nhiệm hữu hạn,15.0,18.0,16.5,21.0,33.0,27.0


# Tiền xử lý dữ liệu

- ở cột Ngành nghề ta thấy có nhiều Ngành nghề khác nhau nhưng chỉ có Ngành nghề xuất hiện đầu tiên là ngành ngề chính, còn những Ngành nghề sau đề là công việc liên quan. Vì vậy ta chỉ lấy Ngành nghề đầu tiên làm Ngành nghề chính. Tìm hiểu xem có tôi đa bao nhiêu ngành nghề iên quan

In [64]:
# Tìm số dâu '/' nhiều nhất trong cột 'Ngành ngề'
max_slash = 0
for i in df['Ngành nghề']:
    if i.count('/') > max_slash:
        max_slash = i.count('/')
print(max_slash)

2


- Như vậy có 1 ngành nghề chính và 2 ngành nghề liên quan. Ta sẽ tách ra thành 3 cột riêng biệt

In [65]:
# Tách cột 'Ngành nghề' thành nhiều cột
df[['Ngành Nghề Chính', 'Nghề Liên quan 1', 'Nghề Liên quan 2']] = df['Ngành nghề'].str.split('/', expand=True, n=2)
df.drop('Ngành nghề', axis=1, inplace=True)
df

,Khu vực tuyển,Thời gian thử việc,Cấp bậc,Yêu cầu giới tính,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Quy mô công ty,Loại công ty,Mức lương thấp nhất,Mức lương cao nhất,Mức lương trung bình,Tuổi thấp nhất,Tuổi cao nhất,Tuổi trung bình,Ngành Nghề Chính,Nghề Liên quan 1,Nghề Liên quan 2
0,Hà Nội,1.6,Chuyên viên- nhân viên,Không yêu cầu,20,Toàn thời gian cố định,Trung cấp,Chưa có kinh nghiệm,Trên 300 người,Công ty cổ phần,7.0,10.0,8.50,22.0,34.0,28.0,Hành chính - Thư ký,Chăm sóc khách hàng,Ngân hàng
1,TP.HCM,1.0,Chuyên viên- nhân viên,Nữ,1,Toàn thời gian cố định,Đại học,2 năm,10 - 150 người,Công ty trách nhiệm hữu hạn,10.0,15.0,12.50,22.0,40.0,31.0,Xuất Nhập Khẩu,Hành chính - Thư ký,Thu mua - Kho Vận - Chuỗi cung ứng
2,Hà Nội,1.0,Chuyên viên- nhân viên,Không yêu cầu,6,Toàn thời gian cố định,Không,Dưới 1 năm,10 - 150 người,Công ty cổ phần,10.0,30.0,20.00,22.0,35.0,28.5,Bán hàng - Kinh doanh,Hành chính - Thư ký,Điện - Điện tử - Điện lạnh
3,Tây Ninh,2.0,Chuyên viên- nhân viên,Nữ,2,Toàn thời gian cố định,Đại học,Chưa có kinh nghiệm,10 - 150 người,Công ty trách nhiệm hữu hạn,10.0,15.0,12.50,22.0,30.0,26.0,Kế toán,Hành chính - Thư ký,Sản xuất - Lắp ráp - Chế biến
4,Bà Rịa - Vũng Tàu,2.0,Chuyên viên- nhân viên,Không yêu cầu,1,Toàn thời gian cố định,Đại học,2 năm,Trên 300 người,Công ty cổ phần,10.0,20.0,15.00,22.0,35.0,28.5,Tài chính - Đầu tư - Chứng Khoán,Kế toán,Hành chính - Thư ký
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5304,"TP.HCM, Long An",1.0,Chuyên viên- nhân viên,Nam,6,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,150 - 300 người,Công ty cổ phần,6.0,6.5,6.25,20.0,25.0,22.5,Lao động phổ thông,None,None
5305,TP.HCM,2.0,Chuyên viên- nhân viên,Nam,2,Toàn thời gian cố định,Không,1 năm,Dưới 10 người,Công ty trách nhiệm hữu hạn,6.0,8.0,7.00,18.0,25.0,21.5,Lao động phổ thông,None,None
5306,Bình Dương,1.8,Chuyên viên- nhân viên,Không yêu cầu,50,Toàn thời gian tạm thời,Không,Chưa có kinh nghiệm,10 - 150 người,Công ty cổ phần,6.0,15.0,10.50,22.0,36.0,29.0,Lao động phổ thông,None,None
5307,TP.HCM,1.0,Cộng tác viên,Không yêu cầu,5,Bán thời gian cố định,Không,Chưa có kinh nghiệm,Trên 300 người,Công ty trách nhiệm hữu hạn,1.0,3.0,2.00,18.0,22.0,20.0,Thực tập sinh,None,None


## Xử lý cột Khu vực tuyển dụng

- Ta thấy 'Khu vực tuyển' Có nhiều giá trị khác nhau là các tỉnh thành đang tuyển nhân sự trong một ô dữ liệu phân tách nhau bỏ dấu ',' nên ta tách ra thành các dòng riêng biệt cho mỗi tỉnh thành.

In [66]:
df_expanded = df.drop('Khu vực tuyển', axis=1).join(
    df['Khu vực tuyển'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('Khu vực tuyển')
)
df_expanded[30:35]

,Thời gian thử việc,Cấp bậc,Yêu cầu giới tính,Số lượng tuyển,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Quy mô công ty,Loại công ty,Mức lương thấp nhất,Mức lương cao nhất,Mức lương trung bình,Tuổi thấp nhất,Tuổi cao nhất,Tuổi trung bình,Ngành Nghề Chính,Nghề Liên quan 1,Nghề Liên quan 2,Khu vực tuyển
27,1.8,Chuyên viên- nhân viên,Không yêu cầu,10,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,150 - 300 người,Công ty trách nhiệm hữu hạn,8.0,15.0,11.5,22.0,37.0,29.5,Vận Tải - Lái xe - Giao nhận,Lao động phổ thông,Hành chính - Thư ký,Gia Lai
27,1.8,Chuyên viên- nhân viên,Không yêu cầu,10,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,150 - 300 người,Công ty trách nhiệm hữu hạn,8.0,15.0,11.5,22.0,37.0,29.5,Vận Tải - Lái xe - Giao nhận,Lao động phổ thông,Hành chính - Thư ký,Quảng Ngãi
27,1.8,Chuyên viên- nhân viên,Không yêu cầu,10,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,150 - 300 người,Công ty trách nhiệm hữu hạn,8.0,15.0,11.5,22.0,37.0,29.5,Vận Tải - Lái xe - Giao nhận,Lao động phổ thông,Hành chính - Thư ký,Phú Yên
27,1.8,Chuyên viên- nhân viên,Không yêu cầu,10,Toàn thời gian cố định,Không,Chưa có kinh nghiệm,150 - 300 người,Công ty trách nhiệm hữu hạn,8.0,15.0,11.5,22.0,37.0,29.5,Vận Tải - Lái xe - Giao nhận,Lao động phổ thông,Hành chính - Thư ký,Bình Định
28,2.0,Chuyên viên- nhân viên,Không yêu cầu,1,Toàn thời gian cố định,Cao đẳng,2 năm,150 - 300 người,Công ty trách nhiệm hữu hạn,13.0,16.0,14.5,22.0,40.0,31.0,Hành chính - Thư ký,Nhân sự,None,Đồng Nai


In [67]:
df = df_expanded.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6786 entries, 0 to 5308
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Thời gian thử việc    6786 non-null   float64
 1   Cấp bậc               6786 non-null   object 
 2   Yêu cầu giới tính     6786 non-null   object 
 3   Số lượng tuyển        6786 non-null   int64  
 4   Hình thức làm việc    6786 non-null   object 
 5   Yêu cầu bằng cấp      6786 non-null   object 
 6   Yêu cầu kinh nghiệm   6786 non-null   object 
 7   Quy mô công ty        6786 non-null   object 
 8   Loại công ty          6786 non-null   object 
 9   Mức lương thấp nhất   6786 non-null   float64
 10  Mức lương cao nhất    6786 non-null   float64
 11  Mức lương trung bình  6786 non-null   float64
 12  Tuổi thấp nhất        6786 non-null   float64
 13  Tuổi cao nhất         6786 non-null   float64
 14  Tuổi trung bình       6786 non-null   float64
 15  Ngành Nghề Chính      6786

# Tìm biến ảnh hưởng đến mức lương

In [68]:
# Chọn những cột 
df_categorical = df.select_dtypes(include=['object'])
df_categorical['Mức lương trùng bình'] = df['Mức lương trung bình']
df_categorical.head(3)

,Cấp bậc,Yêu cầu giới tính,Hình thức làm việc,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Quy mô công ty,Loại công ty,Ngành Nghề Chính,Nghề Liên quan 1,Nghề Liên quan 2,Khu vực tuyển,Mức lương trùng bình
0,Chuyên viên- nhân viên,Không yêu cầu,Toàn thời gian cố định,Trung cấp,Chưa có kinh nghiệm,Trên 300 người,Công ty cổ phần,Hành chính - Thư ký,Chăm sóc khách hàng,Ngân hàng,Hà Nội,8.5
1,Chuyên viên- nhân viên,Nữ,Toàn thời gian cố định,Đại học,2 năm,10 - 150 người,Công ty trách nhiệm hữu hạn,Xuất Nhập Khẩu,Hành chính - Thư ký,Thu mua - Kho Vận - Chuỗi cung ứng,TP.HCM,12.5
2,Chuyên viên- nhân viên,Không yêu cầu,Toàn thời gian cố định,Không,Dưới 1 năm,10 - 150 người,Công ty cổ phần,Bán hàng - Kinh doanh,Hành chính - Thư ký,Điện - Điện tử - Điện lạnh,Hà Nội,20.0


In [70]:
for column in df_categorical.columns[:-1]:  # Loại trừ cột 'Mức lương trung bình'
    formula = f'`Mức lương trung bình` ~ {column}'
    model = ols(formula, data=df_categorical).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    print(f'\nANOVA for {column}:\n', aov_table)

PatsyError: error tokenizing input (maybe an unclosed string?)
    `Mức lương trung bình` ~ {column}
    ^

In [47]:
df_numerical = df.select_dtypes(include=['int64', 'float64']).drop(['Mức lương thấp nhất', 'Mức lương cao nhất'], axis=1)
df_numerical

,Thời gian thử việc,Số lượng tuyển,Mức lương trung bình,Tuổi thấp nhất,Tuổi cao nhất,Tuổi trung bình
0,1.6,20,8.50,22.0,34.0,28.0
1,1.0,1,12.50,22.0,40.0,31.0
2,1.0,6,20.00,22.0,35.0,28.5
3,2.0,2,12.50,22.0,30.0,26.0
4,2.0,1,15.00,22.0,35.0,28.5
...,...,...,...,...,...,...
5304,1.0,6,6.25,20.0,25.0,22.5
5305,2.0,2,7.00,18.0,25.0,21.5
5306,1.8,50,10.50,22.0,36.0,29.0
5307,1.0,5,2.00,18.0,22.0,20.0


# Xây dựng mô hình

In [ ]:
# Chuẩn bị dữ liệu (thay đổi tên cột và đường dẫn dữ liệu tương ứng)
X = df_one_hot_encoded
y = df['Mức lương trung bình']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng và huấn luyện mô hình
models = {
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Extra Trees': ExtraTreesRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor()
}

# Thiết lập các tham số cần tìm kiếm
param_grid = {
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]},
    'Gradient Boosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    'Linear Regression': {},
    'Ridge Regression': {'alpha': [0.1, 1, 10], 'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
    'Lasso Regression': {'alpha': [0.1, 1, 10], 'tol': [0.001, 0.01, 0.1]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'Extra Trees': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    'XGBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
    'LightGBM': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'min_child_samples': [5, 10, 20]}
}


# K-fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search và huấn luyện mô hình
best_models = {}

# Kích thước của ô vẽ
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(16, 16))
axs = axs.flatten()

best_params = []
mses = []
r2s = []
model_names = []

# Grid Search và huấn luyện mô hình
for i, (model_name, model) in enumerate(models.items()):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid[model_name], scoring='neg_mean_squared_error', cv=kf)
    grid_search.fit(X_train, y_train)

    best_models[model_name] = grid_search.best_estimator_

    predictions = grid_search.best_estimator_.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    model_names.append(model_name)
    best_params.append(grid_search.best_params_)
    mses.append(mse)
    r2s.append(r2)

    # Vẽ biểu đồ so sánh giữa giá trị thực tế và dự đoán
    sns.scatterplot(x=y_test, y=predictions, ax=axs[i])
    axs[i].set_title(f'{model_name} - Actual vs Predicted')
    axs[i].set_xlabel('Actual Salary')
    axs[i].set_ylabel('Predicted Salary')

# Tinh chỉnh layout
plt.tight_layout()
plt.show()

In [ ]:
for i, (model_name, model) in enumerate(models.items()):
  print('{} có mean square erros{} & r square erro: {}'.format(model_name,mses[i],r2s[i]))